In [40]:
import pybamm
import time
import numpy as np
import jax
import jax.numpy as jnp

In [41]:
# 1) pip install "pybamm[iree,jax]"  
# 2) then install after "pip install jax[cuda12]" - this upgrades jax to support CUDA12
print("Available devices:", jax.devices())

Available devices: [cuda(id=0)]


In [42]:
# We will want to differentiate our model, so let's define two input parameters
inputs = {
    "Current function [A]": 0.222,
    "Separator porosity": 0.3,
    
}

# Set-up the model
options = {"cell geometry": "arbitrary", "thermal": "lumped"}
model = pybamm.lithium_ion.DFN(options=options)
geometry = model.default_geometry
param = model.default_parameter_values
param.update({key: "[input]" for key in inputs.keys()})
param.process_geometry(geometry)
param.process_model(model)
var = pybamm.standard_spatial_vars
var_pts = {var.x_n: 20, var.x_s: 20, var.x_p: 20, var.r_n: 10, var.r_p: 10}
mesh = pybamm.Mesh(geometry, model.default_submesh_types, var_pts)
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# Use a short time-vector for this example, and declare which variables to track
t_eval = np.linspace(0, 360, 10)
output_variables = [
    "Voltage [V]",
    "Current [A]",
    "Time [min]",
]

# Create the IDAKLU Solver object
idaklu_solver = pybamm.IDAKLUSolver(
    rtol=1e-6,
    atol=1e-6,
    output_variables=output_variables,
)

2024-09-05 13:40:11.384 - [INFO] base_model._build_model(777): Start building Doyle-Fuller-Newman model
2024-09-05 13:40:11.481 - [INFO] base_battery_model.build_model(1042): Finish building Doyle-Fuller-Newman model
2024-09-05 13:40:11.487 - [INFO] parameter_values.process_model(440): Start setting parameters for Doyle-Fuller-Newman model
2024-09-05 13:40:11.805 - [INFO] parameter_values.process_model(509): Finish setting parameters for Doyle-Fuller-Newman model
2024-09-05 13:40:11.809 - [INFO] discretisation.process_model(149): Start discretising Doyle-Fuller-Newman model
2024-09-05 13:40:12.997 - [INFO] discretisation.process_model(251): Finish discretising Doyle-Fuller-Newman model


In [43]:
# This is how we would normally perform a solve using IDAKLU
sim = idaklu_solver.solve(
    model,
    t_eval,
    inputs=inputs,
    calculate_sensitivities=True,
)

# Instead, we Jaxify the IDAKLU solver using similar arguments...
jax_solver = idaklu_solver.jaxify(
    model,
    t_eval,
)

# ... and then obtain a JAX expression for the solve
f = jax_solver.get_jaxpr()
print(f"JAX expression: {f}")

2024-09-05 13:40:13.095 - [INFO] base_solver.solve(744): Start solving Doyle-Fuller-Newman model with IDA KLU solver
2024-09-05 13:40:13.108 - [INFO] base_solver.set_up(117): Start solver set-up
2024-09-05 13:40:13.718 - [INFO] base_solver.set_up(286): Finish solver set-up
2024-09-05 13:40:14.271 - [INFO] base_solver.solve(974): Finish solving Doyle-Fuller-Newman model (the solver successfully reached the end of the integration interval)
2024-09-05 13:40:14.272 - [INFO] base_solver.solve(975): Set-up time: 948.131 ms, Solve time: 214.875 ms (of which integration time: 195.557 ms), Total time: 1.163 s


JAX expression: <function IDAKLUJax._jaxify.<locals>.f at 0x7f2cf5e00040>


In [44]:
# This is how we would normally perform a solve using IDAKLU
sim = idaklu_solver.solve(
    model,
    t_eval,
    inputs=inputs,
    calculate_sensitivities=True,
)

# Instead, we Jaxify the IDAKLU solver using similar arguments...
jax_solver = idaklu_solver.jaxify(
    model,
    t_eval,
)

# ... and then obtain a JAX expression for the solve
f = jax_solver.get_jaxpr()
print(f"JAX expression: {f}")

2024-09-05 13:40:14.478 - [INFO] base_solver.solve(744): Start solving Doyle-Fuller-Newman model with IDA KLU solver
2024-09-05 13:40:14.750 - [INFO] base_solver.solve(974): Finish solving Doyle-Fuller-Newman model (the solver successfully reached the end of the integration interval)
2024-09-05 13:40:14.752 - [INFO] base_solver.solve(975): Set-up time: 21.600 us, Solve time: 269.374 ms (of which integration time: 249.928 ms), Total time: 269.396 ms


JAX expression: <function IDAKLUJax._jaxify.<locals>.f at 0x7f2cf5e000d0>


In [45]:
# Print all output variables, evaluated over a given time vector
data = f(t_eval, inputs)
print(data)

2024-09-05 13:40:14.842 - [INFO] base_solver.solve(744): Start solving Doyle-Fuller-Newman model with IDA KLU solver


2024-09-05 13:40:15.092 - [INFO] base_solver.solve(974): Finish solving Doyle-Fuller-Newman model (the solver successfully reached the end of the integration interval)
2024-09-05 13:40:15.093 - [INFO] base_solver.solve(975): Set-up time: 10.600 us, Solve time: 248.289 ms (of which integration time: 228.796 ms), Total time: 248.300 ms


[[3.81933939e+000 2.22000000e-001 1.15148226e-311]
 [3.81351212e+000 2.22000000e-001 6.66666667e-001]
 [3.81085763e+000 2.22000000e-001 1.33333333e+000]
 [3.80891360e+000 2.22000000e-001 2.00000000e+000]
 [3.80720490e+000 2.22000000e-001 2.66666667e+000]
 [3.80558327e+000 2.22000000e-001 3.33333333e+000]
 [3.80399869e+000 2.22000000e-001 4.00000000e+000]
 [3.80243297e+000 2.22000000e-001 4.66666667e+000]
 [3.80087903e+000 2.22000000e-001 5.33333333e+000]
 [3.79933426e+000 2.22000000e-001 6.00000000e+000]]


In [46]:
# Isolate a single variables
data = jax_solver.get_var("Voltage [V]")(t_eval, inputs)
print(f"Isolating a single variable returns an array of shape {data.shape}")
print(data)

# Isolate two variables from the solver
data = jax_solver.get_vars(
    [
        "Voltage [V]",
        "Current [A]",
    ],
)(t_eval, inputs)
print(f"\nIsolating two variables returns an array of shape {data.shape}")
print(data)

Isolating a single variable returns an array of shape (10,)
[3.81933939 3.81351212 3.81085763 3.8089136  3.8072049  3.80558327
 3.80399869 3.80243297 3.80087903 3.79933426]

Isolating two variables returns an array of shape (10, 2)
[[3.81933939 0.222     ]
 [3.81351212 0.222     ]
 [3.81085763 0.222     ]
 [3.8089136  0.222     ]
 [3.8072049  0.222     ]
 [3.80558327 0.222     ]
 [3.80399869 0.222     ]
 [3.80243297 0.222     ]
 [3.80087903 0.222     ]
 [3.79933426 0.222     ]]


In [47]:
# Calculate the Jacobian matrix (via forward autodiff)
t_start = time.time()
out = jax.jacfwd(f, argnums=1)(t_eval, inputs)
print(f"Jacobian forward method ran in {time.time()-t_start:0.3} secs")
print(out)

# Calculate Jacobian matrix (via backward autodiff)
t_start = time.time()
out = jax.jacrev(f, argnums=1)(t_eval, inputs)
print(f"\nJacobian reverse method ran in {time.time()-t_start:0.3} secs")
print(out)

Jacobian forward method ran in 0.116 secs
{'Current function [A]': Array([[-0.13629603,  1.        ,  0.        ],
       [-0.16363422,  1.        ,  0.        ],
       [-0.17585803,  1.        ,  0.        ],
       [-0.18462944,  1.        ,  0.        ],
       [-0.19225735,  1.        ,  0.        ],
       [-0.19945328,  1.        ,  0.        ],
       [-0.20644869,  1.        ,  0.        ],
       [-0.21332169,  1.        ,  0.        ],
       [-0.22009827,  1.        ,  0.        ],
       [-0.22678395,  1.        ,  0.        ]], dtype=float64), 'Separator porosity': Array([[0.00579553, 0.        , 0.        ],
       [0.00796344, 0.        , 0.        ],
       [0.00951735, 0.        , 0.        ],
       [0.01023533, 0.        , 0.        ],
       [0.01052245, 0.        , 0.        ],
       [0.01063035, 0.        , 0.        ],
       [0.01067032, 0.        , 0.        ],
       [0.01068552, 0.        , 0.        ],
       [0.01069203, 0.        , 0.        ],
       [0

In [48]:
# Isolate the derivate of Voltage with respect to the Current function:
out = jax.jacfwd(f, argnums=1)(t_eval, inputs)
data = jax_solver.get_var(out["Current function [A]"], "Voltage [V]")
print(data)

[-0.13629603 -0.16363422 -0.17585803 -0.18462944 -0.19225735 -0.19945328
 -0.20644869 -0.21332169 -0.22009827 -0.22678395]


In [49]:
# Example evaluation using the `grad` function
t_start = time.time()
data = jax.vmap(
    jax.grad(
        jax_solver.get_var("Voltage [V]"),
        argnums=1,  # take derivative with respect to `inputs`
    ),
    in_axes=(0, None),  # map time over the 0th dimension and do not map inputs
)(t_eval, inputs)
print(f"Gradient method ran in {time.time()-t_start:0.3} secs")
print(data)

Gradient method ran in 0.632 secs
{'Current function [A]': Array([-0.13629603, -0.16363422, -0.17585803, -0.18462944, -0.19225735,
       -0.19945328, -0.20644869, -0.21332169, -0.22009827, -0.22678395],      dtype=float64), 'Separator porosity': Array([0.00579553, 0.00796344, 0.00951735, 0.01023533, 0.01052245,
       0.01063035, 0.01067032, 0.01068552, 0.01069203, 0.01069544],      dtype=float64)}


# use case example

As a use-case example, consider a fitting procedure where we want to compare simulation data against some experimental data. We achieve this by computing the sum-of-squared error (SEE) between the two. Many fitting procedures will converge more quickly (with fewer iterations) if both the value and gradient of the SSE function are provided. By making use of JAX-expressions we can derive these effortlessly.

Note: We do not need to map over time when calling value_and_grad in this example as the sse function returns a scalar (despite taking vector inputs).

In [50]:
# Simulate some experimental data using our original parameter settings
data = sim["Voltage [V]"](t_eval)


# Sum-of-squared errors
def sse(t, inputs):
    modelled = jax_solver.get_var("Voltage [V]")(t_eval, inputs)
    return jnp.sum((modelled - data) ** 2)


# Provide some predicted model inputs (these could come from a fitting procedure)
inputs_pred = {
    "Current function [A]": 0.150,
    "Separator porosity": 0.333,
}

# Get the value and gradient of the SSE function
t_start = time.time()
value, gradient = jax.value_and_grad(sse, argnums=1)(t_eval, inputs_pred)
print(f"Value and gradient computed in {time.time()-t_start:0.3} secs")
print("SSE value: ", value)
print("SSE gradient (wrt each input): ", gradient)

2024-09-05 13:40:23.558 - [INFO] base_solver.solve(744): Start solving Doyle-Fuller-Newman model with IDA KLU solver
2024-09-05 13:40:23.986 - [INFO] base_solver.solve(974): Finish solving Doyle-Fuller-Newman model (the solver successfully reached the end of the integration interval)
2024-09-05 13:40:24.002 - [INFO] base_solver.solve(975): Set-up time: 15.901 us, Solve time: 425.674 ms (of which integration time: 391.195 ms), Total time: 425.690 ms
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for th

Value and gradient computed in 1.24 secs
SSE value:  0.0020770188284553034
SSE gradient (wrt each input):  {'Current function [A]': array(-0.05756411), 'Separator porosity': array(0.00146621)}


'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)


# getting a parameter, x-avg cell volt

this is more so for my own understanding on how the battery simulator api should work when being called to solve for a simulation. right now i dont think it uses output variables but rather simulates all with casadi then filters based on what the user wants... need to double check this.
if so i would need to see if i can call output_variables before using any solver so the results you get from the api call are agnostic to unique solvers

# DFN simulation

In [51]:
import pybamm
import numpy as np

# Define inputs
inputs = {
    "Current function [A]": 0.222,
    "Separator porosity": 0.3,
}

# Set-up the model
options = {"cell geometry": "arbitrary", "thermal": "lumped"}
model = pybamm.lithium_ion.DFN(options=options)
geometry = model.default_geometry
param = model.default_parameter_values
param.update({key: "[input]" for key in inputs.keys()})
param.process_geometry(geometry)
param.process_model(model)
var = pybamm.standard_spatial_vars
var_pts = {var.x_n: 20, var.x_s: 20, var.x_p: 20, var.r_n: 10, var.r_p: 10}
mesh = pybamm.Mesh(geometry, model.default_submesh_types, var_pts)
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# Use a short time-vector for this example
t_eval = np.linspace(0, 360, 10)

# Include the variable of interest in output_variables
output_variables = [
    "Voltage [V]",
    "Current [A]",
    "Time [min]",
    "X-averaged cell temperature [C]" # want to retrieve this and print the output
]

# Create the IDAKLU Solver object
idaklu_solver = pybamm.IDAKLUSolver(rtol=1e-6, atol=1e-6, output_variables=output_variables)

start_time = time.time()

# Perform the simulation
sim = idaklu_solver.solve(
    model,
    t_eval,
    inputs=inputs,
    calculate_sensitivities=True,
)

end_time = time.time()
print(f"Simulation took {end_time - start_time:.2f} seconds.")

# Variable name to extract
var_name = "X-averaged cell temperature [C]"

# Check if the variable is present in the simulation results
if var_name not in sim._variables:
    print(f"Variable '{var_name}' not found in the simulation results.")
else:
    # Extract the variable from the simulation result
    try:
        # Evaluate the variable using the simulation result
        temp_data = sim[var_name](t_eval)
        print("X-averaged cell temperature [C]:")
        print(temp_data)
    except Exception as e:
        print(f"An error occurred while evaluating the variable: {e}")

2024-09-05 13:40:25.062 - [INFO] base_model._build_model(777): Start building Doyle-Fuller-Newman model
2024-09-05 13:40:25.161 - [INFO] base_battery_model.build_model(1042): Finish building Doyle-Fuller-Newman model
2024-09-05 13:40:25.167 - [INFO] parameter_values.process_model(440): Start setting parameters for Doyle-Fuller-Newman model
2024-09-05 13:40:25.425 - [INFO] parameter_values.process_model(509): Finish setting parameters for Doyle-Fuller-Newman model
2024-09-05 13:40:25.436 - [INFO] discretisation.process_model(149): Start discretising Doyle-Fuller-Newman model
2024-09-05 13:40:26.611 - [INFO] discretisation.process_model(251): Finish discretising Doyle-Fuller-Newman model
2024-09-05 13:40:26.614 - [INFO] base_solver.solve(744): Start solving Doyle-Fuller-Newman model with IDA KLU solver
2024-09-05 13:40:26.627 - [INFO] base_solver.set_up(117): Start solver set-up
2024-09-05 13:40:27.383 - [INFO] base_solver.set_up(286): Finish solver set-up
2024-09-05 13:40:28.086 - [INFO

Simulation took 1.48 seconds.
X-averaged cell temperature [C]:
[25.         25.01242858 25.01527114 25.01595353 25.01613004 25.01618514
 25.01620802 25.01622075 25.01622949 25.01623572]


# SPM simulation

In [ ]:
import pybamm
import numpy as np

# Define inputs
inputs = {
    "Current function [A]": 0.222,
    "Separator porosity": 0.3,
}

# Set-up the model
options = {"cell geometry": "arbitrary", "thermal": "lumped"}
model = pybamm.lithium_ion.SPM(options=options)
geometry = model.default_geometry
param = model.default_parameter_values
param.update({key: "[input]" for key in inputs.keys()})
param.process_geometry(geometry)
param.process_model(model)
var = pybamm.standard_spatial_vars
var_pts = {var.x_n: 20, var.x_s: 20, var.x_p: 20, var.r_n: 10, var.r_p: 10}
mesh = pybamm.Mesh(geometry, model.default_submesh_types, var_pts)
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# Use a short time-vector for this example
t_eval = np.linspace(0, 360, 10)

# Include the variable of interest in output_variables
output_variables = [
    "Voltage [V]",
    "Current [A]",
    "Time [min]",
    "X-averaged cell temperature [C]" # want to retrieve this and print the output
]

# Create the IDAKLU Solver object
idaklu_solver = pybamm.IDAKLUSolver(rtol=1e-6, atol=1e-6, output_variables=output_variables)

start_time = time.time()

# Perform the simulation
sim = idaklu_solver.solve(
    model,
    t_eval,
    inputs=inputs,
    calculate_sensitivities=True,
)

end_time = time.time()
print(f"Simulation took {end_time - start_time:.2f} seconds.")

# Variable name to extract
var_name = "X-averaged cell temperature [C]"

# Check if the variable is present in the simulation results
if var_name not in sim._variables:
    print(f"Variable '{var_name}' not found in the simulation results.")
else:
    # Extract the variable from the simulation result
    try:
        # Evaluate the variable using the simulation result
        temp_data = sim[var_name](t_eval)
        print("X-averaged cell temperature [C]:")
        print(temp_data)
    except Exception as e:
        print(f"An error occurred while evaluating the variable: {e}")

Simulation took 0.12 seconds.
X-averaged cell temperature [C]:
[25.         25.01072713 25.01289727 25.01334668 25.01344455 25.01347274
 25.01348601 25.0134948  25.01350122 25.0135058 ]


# SPMe Simulation

In [ ]:
import pybamm
import numpy as np

# Define inputs
inputs = {
    "Current function [A]": 0.222,
    "Separator porosity": 0.3,
}

# Set-up the model
options = {"cell geometry": "arbitrary", "thermal": "lumped"}
model = pybamm.lithium_ion.SPMe(options=options)
geometry = model.default_geometry
param = model.default_parameter_values
param.update({key: "[input]" for key in inputs.keys()})
param.process_geometry(geometry)
param.process_model(model)
var = pybamm.standard_spatial_vars
var_pts = {var.x_n: 20, var.x_s: 20, var.x_p: 20, var.r_n: 10, var.r_p: 10}
mesh = pybamm.Mesh(geometry, model.default_submesh_types, var_pts)
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# Use a short time-vector for this example
t_eval = np.linspace(0, 360, 10)

# Include the variable of interest in output_variables
output_variables = [
    "Voltage [V]",
    "Current [A]",
    "Time [min]",
    "X-averaged cell temperature [C]" # want to retrieve this and print the output
]

# Create the IDAKLU Solver object
idaklu_solver = pybamm.IDAKLUSolver(rtol=1e-6, atol=1e-6, output_variables=output_variables)

start_time = time.time()

# Perform the simulation
sim = idaklu_solver.solve(
    model,
    t_eval,
    inputs=inputs,
    calculate_sensitivities=True,
)

end_time = time.time()
print(f"Simulation took {end_time - start_time:.2f} seconds.")

# Variable name to extract
var_name = "X-averaged cell temperature [C]"

# Check if the variable is present in the simulation results
if var_name not in sim._variables:
    print(f"Variable '{var_name}' not found in the simulation results.")
else:
    # Extract the variable from the simulation result
    try:
        # Evaluate the variable using the simulation result
        temp_data = sim[var_name](t_eval)
        print("X-averaged cell temperature [C]:")
        print(temp_data)
    except Exception as e:
        print(f"An error occurred while evaluating the variable: {e}")

Simulation took 0.51 seconds.
X-averaged cell temperature [C]:
[25.         25.01253857 25.01545068 25.01616518 25.0163572  25.01641667
 25.01643949 25.0164513  25.01645871 25.0164637 ]


In [26]:
#
# Example showing how to create a custom lithium-ion model from submodels
#

import pybamm
import numpy as np

pybamm.set_logging_level("INFO")

# load lithium-ion base model
model = pybamm.lithium_ion.BaseModel(name="my li-ion model")

# set choice of submodels
model.submodels["external circuit"] = pybamm.external_circuit.ExplicitCurrentControl(
    model.param, model.options
)
model.submodels["current collector"] = pybamm.current_collector.Uniform(model.param)
model.submodels["thermal"] = pybamm.thermal.lumped.Lumped(model.param)
model.submodels["porosity"] = pybamm.porosity.Constant(model.param, model.options)
model.submodels["electrolyte diffusion"] = (
    pybamm.electrolyte_diffusion.ConstantConcentration(model.param)
)
model.submodels["electrolyte conductivity"] = (
    pybamm.electrolyte_conductivity.LeadingOrder(model.param)
)

# Loop over negative and positive electrode domains for some submodels
for domain in ["negative", "positive"]:
    model.submodels[f"{domain} active material"] = pybamm.active_material.Constant(
        model.param, domain, model.options
    )
    model.submodels[f"{domain} electrode potential"] = (
        pybamm.electrode.ohm.LeadingOrder(model.param, domain)
    )
    model.submodels[f"{domain} particle"] = pybamm.particle.XAveragedPolynomialProfile(
        model.param,
        domain,
        options={**model.options, "particle": "uniform profile"},
        phase="primary",
    )
    model.submodels[f"{domain} total particle concentration"] = (
        pybamm.particle.TotalConcentration(
            model.param, domain, model.options, phase="primary"
        )
    )

    model.submodels[f"{domain} open-circuit potential"] = (
        pybamm.open_circuit_potential.SingleOpenCircuitPotential(
            model.param,
            domain,
            "lithium-ion main",
            options=model.options,
            phase="primary",
        )
    )
    model.submodels[f"{domain} interface"] = pybamm.kinetics.InverseButlerVolmer(
        model.param, domain, "lithium-ion main", options=model.options
    )
    model.submodels[f"{domain} interface utilisation"] = (
        pybamm.interface_utilisation.Full(model.param, domain, model.options)
    )
    model.submodels[f"{domain} interface current"] = (
        pybamm.kinetics.CurrentForInverseButlerVolmer(
            model.param, domain, "lithium-ion main"
        )
    )
    model.submodels[f"{domain} surface potential difference [V]"] = (
        pybamm.electrolyte_conductivity.surface_potential_form.Explicit(
            model.param, domain, model.options
        )
    )
    model.submodels[f"{domain} particle mechanics"] = (
        pybamm.particle_mechanics.NoMechanics(model.param, domain, model.options)
    )
    model.submodels[f"{domain} sei"] = pybamm.sei.NoSEI(
        model.param, domain, model.options
    )
    model.submodels[f"{domain} sei on cracks"] = pybamm.sei.NoSEI(
        model.param, domain, model.options, cracks=True
    )
    model.submodels[f"{domain} lithium plating"] = pybamm.lithium_plating.NoPlating(
        model.param, domain
    )

# build model
model.build_model()

# create geometry
geometry = pybamm.battery_geometry()

# process model and geometry
#LFP BPX
param = pybamm.ParameterValues.create_from_bpx(r'/mnt/c/Users/MVeerasi.ANALOG/Documents/InternshipFY24/Battery-Simulator/BatterySimulator/Models/LFP/lfp_18650_cell_BPX.json')
#NMC BPX
#param = pybamm.ParameterValues.create_from_bpx(r'/mnt/c/Users/MVeerasi.ANALOG/Documents/InternshipFY24/Battery-Simulator/BatterySimulator/Models/NMC/nmc_pouch_cell_BPX.json')
param.process_model(model)
param.process_geometry(geometry)

# set mesh
# Note: li-ion base model has defaults for mesh and var_pts
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise model
# Note: li-ion base model has default spatial methods
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# solve model
t_eval = np.linspace(0, 3600, 100)
solver = pybamm.IDAKLUSolver()
sim = pybamm.Simulation(model, parameter_values=param, solver=solver)
solution = solver.solve(model=model, t_eval=t_eval)

# plot
# plot = pybamm.QuickPlot(solution)
# plot.dynamic_plot()
output_variables = ["Voltage [V]", "Current [A]", 'X-averaged cell temperature [C]']
solution.plot(output_variables=output_variables)

2024-09-05 14:06:11.318 - [INFO] base_model._build_model(777): Start building my li-ion model


2024-09-05 14:06:11.393 - [INFO] base_battery_model.build_model(1042): Finish building my li-ion model
2024-09-05 14:06:11.474 - [INFO] parameter_values.process_model(440): Start setting parameters for my li-ion model
2024-09-05 14:06:11.531 - [INFO] parameter_values.process_model(509): Finish setting parameters for my li-ion model
2024-09-05 14:06:11.534 - [INFO] discretisation.process_model(149): Start discretising my li-ion model
2024-09-05 14:06:11.681 - [INFO] discretisation.process_model(251): Finish discretising my li-ion model
2024-09-05 14:06:11.684 - [INFO] base_solver.solve(744): Start solving my li-ion model with IDA KLU solver
2024-09-05 14:06:11.686 - [INFO] base_solver.set_up(117): Start solver set-up
2024-09-05 14:06:11.710 - [INFO] base_solver.set_up(286): Finish solver set-up
2024-09-05 14:06:11.722 - [INFO] base_solver.solve(974): Finish solving my li-ion model (the solver successfully reached the end of the integration interval)
2024-09-05 14:06:11.723 - [INFO] base

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [14]:
import pybamm
import os
import pandas as pd
from pybamm.expression_tree.functions import exp
os.chdir(pybamm.__path__[0]+'/..')
import numpy as np
import timeit
from scipy import interpolate

pybamm.set_logging_level("INFO")

experiment = pybamm.Experiment(
    [
        (
            "Discharge at C/5 for 10 hours or until 2.5 V",
        )
    ]
)

options = {"thermal": "lumped"}
model = pybamm.lithium_ion.DFN(options=options)
parameter_values = pybamm.ParameterValues.create_from_bpx(r'/mnt/c/Users/MVeerasi.ANALOG/Documents/InternshipFY24/Battery-Simulator/BatterySimulator/Models/LFP/lfp_18650_cell_BPX.json')

t_eval = np.linspace(0, 3600, 100)

solver = pybamm.IDAKLUSolver()
sim = pybamm.Simulation(model,parameter_values=parameter_values, solver=solver, experiment=experiment)
sim.solve()
output_variables = ["Voltage [V]", "Current [A]", 'X-averaged cell temperature [C]']
sim.plot(output_variables=output_variables)

2024-09-05 14:20:00.938 - [INFO] base_model._build_model(777): Start building Doyle-Fuller-Newman model


2024-09-05 14:20:01.013 - [INFO] base_battery_model.build_model(1042): Finish building Doyle-Fuller-Newman model
2024-09-05 14:20:01.088 - [INFO] callbacks.on_experiment_start(162): Start running experiment
2024-09-05 14:20:01.089 - [INFO] parameter_values.process_model(440): Start setting parameters for Doyle-Fuller-Newman model
2024-09-05 14:20:01.182 - [INFO] parameter_values.process_model(509): Finish setting parameters for Doyle-Fuller-Newman model
2024-09-05 14:20:01.185 - [INFO] discretisation.process_model(149): Start discretising Doyle-Fuller-Newman model
2024-09-05 14:20:01.638 - [INFO] discretisation.process_model(251): Finish discretising Doyle-Fuller-Newman model
2024-09-05 14:20:01.639 - [NOTICE] callbacks.on_cycle_start(170): Cycle 1/1 (60.400 us elapsed) --------------------
2024-09-05 14:20:01.640 - [NOTICE] callbacks.on_step_start(178): Cycle 1/1, step 1/1: Discharge at C/5 for 10 hours or until 2.5 V
2024-09-05 14:20:02.151 - [INFO] base_solver.set_up(117): Start sol

interactive(children=(FloatSlider(value=0.0, description='t', max=5.156847598770717, step=0.051568475987707176…